In [1]:
import torch
import ray
import gym
from IPython import display
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [1]:
from ray.rllib.agents import dqn

trainer = dqn.DQNTrainer(env='Breakout-v0')
for i in range(100):
    print(trainer.train()['episode_reward_mean'])

Instructions for updating:
non-resource variables are not supported in the long term
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Install gputil for GPU system monitoring.


Instructions for updating:
Prefer Variable.assign which has equivalent behavior in 2.X.
2.3333333333333335
1.0
1.1538461538461537
1.3529411764705883
1.4285714285714286
1.2592592592592593
1.2903225806451613
1.3055555555555556
1.3
1.2666666666666666
1.24
1.2777777777777777
1.2372881355932204
1.2857142857142858
1.2686567164179106
1.2361111111111112
1.205128205128205
1.2195121951219512
1.255813953488372
1.2173913043478262
1.1734693877551021
1.16
1.16
1.19
1.15
1.11
1.13
1.05
1.0
0.98
0.94
0.93
0.88
0.88
0.91
0.89
0.89
0.91
0.82
0.8
0.75
0.73
0.8
0.81
0.84
0.87
0.95
0.96
1.07
1.12
1.18
1.2
1.2
1.26
1.34
1.34
1.34
1.35
1.37
1.44
1.5
1.59
1.66
1.71
1.83
1.95
1.98
2.15
2.2
2.29
2.39
2.46
2.54
2.57
2.61
2.77
2.8
2.83
2.98
3.03
3.08
3.16
3.23
3.29
3.43
3.58
3.72
3.84
3.89
3.93
4.07
4.16
4.26
4.36
4.48
4.51
4.66
4.7
4.78
4.86
